In [1]:
#to load as many reviews as you want
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

def load_reviews(url, driver):
    time.sleep(2)

    #going to the page with all the reviews
    driver.implicitly_wait(8)
    loadMore = driver.find_element(By.CSS_SELECTOR, "a.Button.Button--transparent.Button--small")
    reviews_page = loadMore.get_attribute("href")
    driver.get(reviews_page) #transfers us to the page with all the reviews

    popUp = driver.find_element(By.CSS_SELECTOR, "button.Button.Button--transparent.Button--small.Button--rounded")
    #checking if pop-up appears
    if popUp:
        driver.implicitly_wait(3)
        popUp.click()
        popUp = driver.find_element(By.CSS_SELECTOR, "button.Button.Button--transparent.Button--small.Button--rounded")
        if popUp: #sometimes it appears twice, so it has to be clicked once more
            popUp.click()


    showMoreBtn = driver.find_element(By.CSS_SELECTOR, "button.Button.Button--secondary.Button--small")
    count = 0
    while(count < 10): #you can change the limit to however many times you want the page to load more reviews, I limited it to 10
        showMoreBtn = driver.find_element(By.CSS_SELECTOR, "button.Button.Button--secondary.Button--small")
        driver.implicitly_wait(3)
        driver.execute_script("arguments[0].click();", showMoreBtn)
        print("Clicked ",count, " times")
        count += 1
        time.sleep(2)


In [2]:
#to get reviews
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

#print statements throughout this method are to help show you where the program is at with its execution

def get_reviews(url):
    page = requests.get(url)
    d = BeautifulSoup(page.text, 'html.parser')
    
    driver = webdriver.Chrome()
    driver.get(url)
    driver.maximize_window()
    
    load_reviews(url, driver)
    
    #retrieving the reviews and shelf-statuses
    reviews_in_page = driver.find_elements(By.CLASS_NAME, 'Formatted')
    time.sleep(2)
    shelfStat = driver.find_elements(By.CLASS_NAME, 'ShelfStatus') #ie rating, how many stars out of 5
    time.sleep(2)
    print("Got reviews and shelf statuses")
    
    #cleaning the data as some people do not rate books and keep it shelved as Read or Currently Reading
    ratingStars = []
    print("Before loop")
    for element in shelfStat:
        try:
            rating = element.find_element(By.CSS_SELECTOR, 'span.RatingStars.RatingStars__small')
            stars = rating.get_attribute('aria-label')
            ratingStars.append(stars[7])
        except:
            ratingStars.append(0) #if reviewers hasn't rated it, just mark it as rated 0 stars
            
    #extracting text from reviews data
    reviewsList = list(map(lambda x: x.text, reviews_in_page))
    print("Got here")
    
    #defining data frame
    revRate = pd.DataFrame({'Rating': ratingStars, 'Review': reviewsList})
    revRate['Review'] = revRate['Review'].apply(lambda x: x.replace('\n', '')) 
            
    return revRate

In [3]:
page_url = 'https://www.goodreads.com/book/show/10429045-shatter-me?ac=1&from_search=true&qid=1NvrHu83wW&rank=1'
reviews = get_reviews(page_url)
reviews

Clicked  0  times
Clicked  1  times
Clicked  2  times
Clicked  3  times
Clicked  4  times
Clicked  5  times
Clicked  6  times
Clicked  7  times
Clicked  8  times
Clicked  9  times
Got reviews and ratings
Before loop
Got here


,Rating,Review
0,1,"This is not a dystopia, it is a romance. This ..."
1,1,I know many found this book to have a very uni...
2,0,MY PRECIOUS IS NOW SIGNED BY TAHEREH!!! I can ...
3,2,Juliette describing Adam’s eyes“...perfect sha...
4,1,I deserve a medal for finishing this book.Like...
...,...,...
295,4,Beautifully written with lyrical and poetic ph...
296,3,"I didn't hate this, but I didn't love it? Juli..."
297,4,Nota final: 4.5/5⭐️⭐️⭐️. Hasta que terminé de ...
298,2,"⭐2 stars⭐""Aw, you trust me?""""As long as I have..."


In [4]:
reviews.to_csv('ShatteMe_Reviews_10pages.csv') #converting data to CSV file